# Enriching PPP Originating Lender Data by adding Financial Data

In [1]:
import requests
import urllib.parse
import pandas as pd
import numpy as np
import matplotlib

# Uncomment to display all columns
# pd.set_option('display.max_columns', None)

In [2]:
# Reading PPP Dataset
PPP_df = pd.read_csv('../data/interim/PPP_scoped.csv')

In [3]:
PPP_df.describe(include='all')

,Unnamed: 0.1,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,OriginatingLenderLocationID,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit,ForgivenessAmount,ForgivenessDate,DateApproved_dt
count,5.130742e+06,5.130742e+06,5.130742e+06,5130742,5.130732e+06,5130742,5130714,5130554,5130556,5130595,...,5.130732e+06,5130732,5130732,5130732,5130742,5130742,183512,4.908368e+06,4908368,5130742
unique,NaN,NaN,NaN,121,NaN,2,4980513,4711740,64457,58,...,NaN,4606,3013,55,3,3,1,NaN,610,121
top,NaN,NaN,NaN,05/01/2020,NaN,PPP,FIRST UNITED METHODIST CHURCH,PO BOX,NEW YORK,CA,...,NaN,"Bank of America, National Association",CHARLOTTE,OH,Unanswered,Unanswered,Y,NaN,01/07/2021,2020-05-01
freq,NaN,NaN,NaN,800436,NaN,5130729,477,578,38666,611271,...,NaN,343488,424062,587351,3469094,3854363,183512,NaN,150891,800436
mean,5.390788e+06,4.505061e+05,5.479823e+09,NaN,5.730732e+02,NaN,NaN,NaN,NaN,NaN,...,1.246282e+05,NaN,NaN,NaN,NaN,NaN,NaN,1.024076e+05,NaN,NaN
std,3.442762e+06,2.647076e+05,2.557538e+09,NaN,2.547571e+02,NaN,NaN,NaN,NaN,NaN,...,1.550522e+05,NaN,NaN,NaN,NaN,NaN,NaN,3.477849e+05,NaN,NaN
min,0.000000e+00,0.000000e+00,1.000007e+09,NaN,1.010000e+02,NaN,NaN,NaN,NaN,NaN,...,2.000000e+01,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e-02,NaN,NaN
25%,2.383614e+06,2.165252e+05,3.305347e+09,NaN,4.050000e+02,NaN,NaN,NaN,NaN,NaN,...,2.959900e+04,NaN,NaN,NaN,NaN,NaN,NaN,1.075541e+04,NaN,NaN
50%,5.128188e+06,4.511120e+05,5.487228e+09,NaN,5.080000e+02,NaN,NaN,NaN,NaN,NaN,...,5.781100e+04,NaN,NaN,NaN,NaN,NaN,NaN,2.362628e+04,NaN,NaN
75%,8.489814e+06,6.805480e+05,7.624295e+09,NaN,7.670000e+02,NaN,NaN,NaN,NaN,NaN,...,1.215360e+05,NaN,NaN,NaN,NaN,NaN,NaN,7.104430e+04,NaN,NaN


## Getting and Enriching Originating Lenders

### Grouping loan data by Originating Lenders

In [4]:
# Aggregating Originating Lender Data

lenders_df = PPP_df.groupby(['OriginatingLender','OriginatingLenderLocationID', 'OriginatingLenderCity','OriginatingLenderState']).agg({'CurrentApprovalAmount':['sum'], 'ForgivenessAmount': ['sum']}).reset_index().sort_values(by=('CurrentApprovalAmount', 'sum'),ascending=False)
lenders_df.columns = lenders_df.columns.get_level_values(0)
lenders_df['ForgivnessPct'] = lenders_df.ForgivenessAmount / lenders_df.CurrentApprovalAmount

lenders_df.head(10)



,OriginatingLender,OriginatingLenderLocationID,OriginatingLenderCity,OriginatingLenderState,CurrentApprovalAmount,ForgivenessAmount,ForgivnessPct
2606,"JPMorgan Chase Bank, National Association",48270.0,COLUMBUS,OH,2.934407e+10,2.808165e+10,0.956978
347,"Bank of America, National Association",9551.0,CHARLOTTE,NC,2.547771e+10,2.361248e+10,0.926790
3303,"PNC Bank, National Association",44449.0,WILMINGTON,DE,1.299645e+10,1.250585e+10,0.962252
4766,Truist Bank,225134.0,CHARLOTTE,NC,1.254766e+10,1.232450e+10,0.982215
5023,"Wells Fargo Bank, National Association",12096.0,SIOUX FALLS,SD,1.028036e+10,9.760728e+09,0.949454
2861,Manufacturers and Traders Trust Company,46391.0,BUFFALO,NY,9.347372e+09,9.194415e+09,0.983636
4092,"TD Bank, National Association",29805.0,WILMINGTON,DE,8.549197e+09,8.224545e+09,0.962025
4510,The Huntington National Bank,57328.0,COLUMBUS,OH,8.340394e+09,8.135444e+09,0.975427
2668,KeyBank National Association,56102.0,CLEVELAND,OH,8.203828e+09,7.866560e+09,0.958889
305,BMO Harris Bank National Association,21442.0,CHICAGO,IL,7.818665e+09,7.579577e+09,0.969421


### Enriching lenders with Top Holding Company data (pre-requisite for stock data)

In [5]:
# Getting and prepping institutions data
df_institutions = pd.read_csv('../data/raw/institution_data/institutions.csv')
# Removing non-current status data
df_institutions['end_date_dt'] = pd.to_datetime(df_institutions['ENDEFYMD'],errors='coerce')
df_institutions_effective = df_institutions[df_institutions['ENDEFYMD'].str.contains('12/31/9999')]
# Keeping only relevant columns, and renaming to match PPP indices
df_institutions_effective = df_institutions_effective[['NAME','CITY','NAMEHCR','UNINUM','RSSDHCR']]
df_institutions_effective.rename({'NAME': 'OriginatingLender', 'CITY': 'OriginatingLenderCity'}, axis=1, inplace=True)
# Removing rows with null NAMEHCR
df_institutions_effective = df_institutions_effective[df_institutions_effective.NAMEHCR.notna()] 
# Standardizing indices
df_institutions_effective['OriginatingLenderCity'] = df_institutions_effective['OriginatingLenderCity'].str.upper()
df_institutions_effective['OriginatingLender'] = df_institutions_effective['OriginatingLender'].str.upper()
# Setting join indices
df_institutions_effective.set_index(['OriginatingLender', 'OriginatingLenderCity'], inplace=True)

# Prepping Lender PPP data
lenders_effective_df = lenders_df.copy()
lenders_effective_df['OriginatingLenderCity'] = lenders_effective_df['OriginatingLenderCity'].str.upper()
lenders_effective_df['OriginatingLender'] = lenders_effective_df['OriginatingLender'].str.upper()
lenders_effective_df.set_index(['OriginatingLender','OriginatingLenderCity'], inplace=True)

# Getting alternative name data
df_alt_names = pd.read_csv('../data/raw/institution_data/institutions_names.csv')
# Keeping only relevant columns, and prepping index
df_alt_names = df_alt_names[['#ID_RSSD','NM_LGL','NM_SHORT',]]
df_alt_names.rename({'#ID_RSSD': 'RSSDHCR'}, axis=1, inplace=True)
df_alt_names.set_index(['RSSDHCR'], inplace=True)
df_alt_names

# First Join: PPP Lenders vs institutions
join_df = lenders_effective_df.join(df_institutions_effective, how='left')
join_df.reset_index(inplace=True)
join_df.set_index('RSSDHCR', inplace=True)

# Second Join: PPP Lenders vs institutions_names
lenders_enriched_df = join_df.join(df_alt_names, how='left', )
lenders_enriched_df.reset_index(inplace=True)
lenders_enriched_df.sort_values('CurrentApprovalAmount', ascending=False)

C:\Users\amarq\AppData\Local\Temp\ipykernel_9644\3712898960.py:2: DtypeWarning: Columns (93,94,95,96,97) have mixed types. Specify dtype option on import or set low_memory=False.
  df_institutions = pd.read_csv('../data/raw/institution_data/institutions.csv')


,RSSDHCR,OriginatingLender,OriginatingLenderCity,OriginatingLenderLocationID,OriginatingLenderState,CurrentApprovalAmount,ForgivenessAmount,ForgivnessPct,NAMEHCR,UNINUM,NM_LGL,NM_SHORT
25,1039502.0,"JPMORGAN CHASE BANK, NATIONAL ASSOCIATION",COLUMBUS,48270.0,OH,2.934407e+10,2.808165e+10,0.956978,JPMORGAN CHASE&CO,417.0,JPMORGAN CHASE & CO. ...,JPMORGAN CHASE & CO
284,1073757.0,"BANK OF AMERICA, NATIONAL ASSOCIATION",CHARLOTTE,9551.0,NC,2.547771e+10,2.361248e+10,0.926790,BANK OF AMERICA CORP,2238.0,BANK OF AMERICA CORPORATION ...,BANK OF AMER CORP
255,1069778.0,"PNC BANK, NATIONAL ASSOCIATION",WILMINGTON,44449.0,DE,1.299645e+10,1.250585e+10,0.962252,PNC FINL SERVICES GROUP INC,4287.0,"PNC FINANCIAL SERVICES GROUP, INC., THE ...",PNC FNCL SVC GROUP
285,1074156.0,TRUIST BANK,CHARLOTTE,225134.0,NC,1.254766e+10,1.232450e+10,0.982215,TRUIST FINANCIAL CORP,6300.0,TRUIST FINANCIAL CORPORATION ...,TRUIST FC
628,1120754.0,"WELLS FARGO BANK, NATIONAL ASSOCIATION",SIOUX FALLS,12096.0,SD,1.028036e+10,9.760728e+09,0.949454,WELLS FARGO&COMPANY,2239.0,WELLS FARGO & COMPANY ...,WELLS FARGO & CO
...,...,...,...,...,...,...,...,...,...,...,...,...
4718,NaN,SOUTH SIDE COMMUNITY FCU,CHICAGO,347974.0,IL,6.427000e+03,6.597510e+03,1.026530,NaN,NaN,NaN,NaN
4380,NaN,NEW BEDFORD CU,NEW BEDFORD,98440.0,MA,5.824000e+03,5.873830e+03,1.008556,NaN,NaN,NaN,NaN
4454,NaN,ONE VISION FCU,CLARKSVILLE,96718.0,IN,5.468750e+03,5.529210e+03,1.011056,NaN,NaN,NaN,NaN
3174,NaN,705 FCU,LAFAYETTE,97794.0,LA,4.722500e+03,4.748740e+03,1.005556,NaN,NaN,NaN,NaN


### Enriching lender data with CERT FDIC Identifier (pre-requisite for balance sheet data)

In [6]:
def get_cert_from_lender_name(lender_name):
    lender_name = urllib.parse.quote(lender_name)
    lender_name = lender_name.rstrip(".")
    URL = f'https://banks.data.fdic.gov/api/institutions?filters=NAME:"{lender_name}"&fields=CERT&sort_by=OFFICES&sort_order=DESC&limit=10&offset=0&format=json&download=false&filename=data_file'
    # headers = {"accept:": "application/json"}
    r = requests.get(URL)
    # print(lender_name, r.json())
    if 'errors' in r.json():
        return None
    if len(r.json()['data']) >0:
        return r.json()['data'][0]['data']['CERT']
    lender_name = lender_name.replace(",", "")
    URL = f'https://banks.data.fdic.gov/api/institutions?filters=NAME:"{lender_name}"&fields=CERT&sort_by=OFFICES&sort_order=DESC&limit=10&offset=0&format=json&download=false&filename=data_file'
    r = requests.get(URL)
    if len(r.json()['data']) >0:
         return r.json()['data'][0]['data']['CERT']
    return None

lenders_enriched_df['CERT'] = lenders_enriched_df['OriginatingLender'].map(get_cert_from_lender_name)

In [7]:
lenders_enriched_df.to_csv('../data/interim/lender_data.csv')

## Getting Balance Sheet Data from FDIC API

In [8]:
def get_balance_sheet_from_cert(row):
    cert = row.CERT
    if cert is None:
        return (None, None, None, None, None, None, None, None, None)
    URL = f"https://banks.data.fdic.gov/api/financials?filters=CERT%3A{cert}%20AND%20ACTIVE%3A1%20AND%20REPDTE%3A20200930&fields=CERT%2CREPDTE%2CASSET%2CLIAB%2CEQ%2CLIABEQ%2CEQCS%2CEQPP%2CSUBLLPF%2CEQCTRSTX&limit=10&offset=0&format=json&download=false"
    r = requests.get(URL)
    # print(r.json()['data'][0]['data']['SUBLLPF'])
    if 'errors' in r.json():
        return None
    if len(r.json()['data']) == 0:
        return None
    SUBLLPF = r.json()['data'][0]['data']['SUBLLPF']
    EQCTRSTX = r.json()['data'][0]['data']['EQCTRSTX']
    LIABEQ = r.json()['data'][0]['data']['LIABEQ']
    ASSET = r.json()['data'][0]['data']['ASSET']
    CERT = r.json()['data'][0]['data']['CERT']
    EQCS = r.json()['data'][0]['data']['EQCS']
    EQ = r.json()['data'][0]['data']['EQ']
    EQPP = r.json()['data'][0]['data']['EQPP']
    LIAB = r.json()['data'][0]['data']['LIAB']
    return {'CERT':CERT, 'ASSET': ASSET, 'LIABEQ':LIABEQ, 'LIAB':LIAB, 'EQ':EQ, 'EQCS':EQCS, 'EQPP':EQPP, 'SUBLLPF':SUBLLPF,'EQCTRSTX': EQCTRSTX}
 
balance_sheet_df = lenders_enriched_df.apply(get_balance_sheet_from_cert, axis='columns', result_type='expand')

In [9]:
balance_sheet_df[balance_sheet_df.CERT.notna()].to_csv('../data/interim/balance_sheet.csv')

# Getting Stock Data using top holding company name

###  Getting tickers

In [10]:
lender_holding_df = lenders_enriched_df.copy()
tickers = pd.read_json('../data/raw/institution_data/iex_tickers.json')
tickers['name'] = tickers['name'].str.upper()
# tickers = tickers[~tickers.symbol.str.contains('.')]
import re
pat = f'({"|".join([re.escape(ticker) for ticker in tickers.name])})'
lender_holding_df.insert(0, 'name_LGL', lender_holding_df['NM_LGL'].str.extract(pat, expand=False,flags=re.IGNORECASE))
lender_holding_df.insert(0, 'name_SHORT', lender_holding_df['NM_SHORT'].str.extract(pat, expand=False,flags=re.IGNORECASE))
lender_holding_df.insert(0, 'name_HCR', lender_holding_df['NAMEHCR'].str.extract(pat, expand=False,flags=re.IGNORECASE))

def check_names(row):
    if pd.notnull(row.name_LGL):
        return(row.name_LGL)
    elif pd.notnull(row.name_SHORT):
        return(row.name_SHORT)
    else:
        return(row.name_HCR)

lender_holding_df['combined_name'] = lender_holding_df.apply(check_names, axis=1)
lender_holding_df

,name_HCR,name_SHORT,name_LGL,RSSDHCR,OriginatingLender,OriginatingLenderCity,OriginatingLenderLocationID,OriginatingLenderState,CurrentApprovalAmount,ForgivenessAmount,ForgivnessPct,NAMEHCR,UNINUM,NM_LGL,NM_SHORT,CERT,combined_name
0,NaN,NaN,NaN,1020395.0,CCB COMMUNITY BANK,ANDALUSIA,26.0,AL,2.142037e+07,2.122405e+07,0.990835,SOUTHERN NATIONAL CORP,10739.0,SOUTHERN NATIONAL CORPORATION ...,SOUTHERN NAT CORP,16595.0,NaN
1,NaN,NaN,NaN,1020582.0,WOODTRUST BANK,WISCONSIN RAPIDS,77051.0,WI,3.154369e+07,3.183455e+07,1.009221,WOODTRUST FINANCIAL CORP,3599.0,WOODTRUST FINANCIAL CORPORATION ...,WOODTRUST FC,5335.0,NaN
2,NaN,NaN,NaN,1020667.0,NORTHWEST BANK & TRUST COMPANY,DAVENPORT,24824.0,IA,2.696372e+07,2.682464e+07,0.994842,NORTHWEST INVESTMENT CORP,10071.0,NORTHWEST INVESTMENT CORP. ...,NORTHWEST INV CORP,15830.0,NaN
3,NaN,NaN,NaN,1020676.0,AMALGAMATED BANK OF CHICAGO,CHICAGO,20494.0,IL,3.049178e+06,3.036049e+06,0.995694,AMALGAMATED INVESTMENTS CO,581.0,AMALGAMATED INVESTMENTS COMPANY ...,AMALGAMATED INV CO,903.0,NaN
4,NaN,NaN,NaN,1020854.0,FIRSTBANK OF NEBRASKA,WAHOO,42693.0,NE,1.042674e+07,1.040137e+07,0.997566,BANK MANAGEMENT INC,3718.0,"BANK MANAGEMENT, INC. ...",BANK MGMT,5486.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5147,NaN,NaN,NaN,NaN,WYOMING CAPITAL ACCESS,CASPER,188132.0,WY,8.744045e+05,8.820143e+05,1.008703,NaN,NaN,NaN,NaN,NaN,NaN
5148,NaN,NaN,NaN,NaN,YANTIS FCU,YANTIS,106720.0,TX,2.850380e+05,2.871083e+05,1.007263,NaN,NaN,NaN,NaN,NaN,NaN
5149,NaN,NaN,NaN,NaN,"YOSEMITE FARM CREDIT, ACA",TURLOCK,528996.0,CA,3.998391e+07,3.904683e+07,0.976564,NaN,NaN,NaN,NaN,NaN,NaN
5150,NaN,NaN,NaN,NaN,ZEAL CU,LIVONIA,98804.0,MI,3.228205e+04,3.278898e+04,1.015703,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
lender_holding_df['NM_LGL'] = lender_holding_df['NM_LGL'].str.strip()
lender_holding_df_all = lender_holding_df.copy()
lender_holding_df = lender_holding_df[lender_holding_df.NM_LGL.notna()]

import difflib

def fuzzy_merge(df1, df2, left_on, right_on, how='inner', cutoff=0.825):
    df_other= df1.copy()
    df_other[right_on] = [get_closest_match(x, df2[right_on], cutoff) 
                         for x in df_other[left_on]]
    return df_other.merge(df2, on=right_on, how=how)

def get_closest_match(x, other, cutoff):
    matches = difflib.get_close_matches(x, other, cutoff=cutoff)
    return matches[0] if matches else None

# lender_holding_df.reset_index(inplace=True)

final = fuzzy_merge(df1 = lender_holding_df, df2 = tickers, right_on= 'name', left_on='NM_LGL')
final


# df_other= lender_holding_df.copy()
# df_other.name = [get_closest_match(x, tickers.name.astype("string"),0.76) for x in df_other.NM_LGL.astype("string")]
# # return df_other.merge(df2, on=right_on, how=how)
# df_other.name 

,name_HCR,name_SHORT,name_LGL,RSSDHCR,OriginatingLender,OriginatingLenderCity,OriginatingLenderLocationID,OriginatingLenderState,CurrentApprovalAmount,ForgivenessAmount,...,exchangeSegmentName,date,type,iexId,region,currency,isEnabled,figi,cik,lei
0,NaN,NaN,NaN,1020582.0,WOODTRUST BANK,WISCONSIN RAPIDS,77051.0,WI,3.154369e+07,3.183455e+07,...,New York Stock Exchange Inc,2023-05-16,cs,IEX_4758354C4A432D52,US,USD,True,BBG000BYYLS8,92230.0,549300DRQQI75D2JP341
1,NaN,NaN,NaN,1064278.0,"INTRUST BANK, NATIONAL ASSOCIATION",WICHITA,26663.0,KS,5.037491e+08,4.991787e+08,...,New York Stock Exchange Inc,2023-05-16,cs,IEX_4758354C4A432D52,US,USD,True,BBG000BYYLS8,92230.0,549300DRQQI75D2JP341
2,NaN,NaN,NaN,1067514.0,IMPACT BANK,WELLINGTON,25733.0,KS,4.688252e+06,4.690073e+06,...,New York Stock Exchange Inc,2023-05-16,cs,IEX_4758354C4A432D52,US,USD,True,BBG000BYYLS8,92230.0,549300DRQQI75D2JP341
3,NaN,NaN,TRUIST FINANCIAL CORPORATION,1074156.0,TRUIST BANK,CHARLOTTE,225134.0,NC,1.254766e+10,1.232450e+10,...,New York Stock Exchange Inc,2023-05-16,cs,IEX_4758354C4A432D52,US,USD,True,BBG000BYYLS8,92230.0,549300DRQQI75D2JP341
4,NaN,NaN,NaN,1128040.0,BANKNORTH,ARTHUR,52663.0,ND,1.634180e+07,1.622817e+07,...,New York Stock Exchange Inc,2023-05-16,cs,IEX_4758354C4A432D52,US,USD,True,BBG000BYYLS8,92230.0,549300DRQQI75D2JP341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1042,NaN,NaN,NaN,5589939.0,"PYRAMAX BANK, FSB",GREENFIELD,81337.0,WI,3.026980e+07,3.033543e+07,...,Nasdaq Capital Market,2023-05-16,cs,IEX_433633544A502D52,US,USD,True,BBG011S2J821,1847360.0,None
1043,NaN,NaN,BLUE FOUNDRY BANCORP,5650781.0,BLUE FOUNDRY BANK,RUTHERFORD,85732.0,NJ,5.719620e+07,5.680090e+07,...,Nasdaq Ngs Global Select Market,2023-05-16,cs,None,US,USD,True,BBG00ZLHY9C1,1846017.0,5493008KPK5NJT0FNK56
1044,NaN,NaN,NaN,5654323.0,NORTH SHORE TRUST AND SAVINGS,WAUKEGAN,77502.0,IL,8.647189e+05,8.610842e+05,...,Nasdaq Capital Market,2023-05-16,cs,None,US,USD,True,BBG012QGT8H3,NaN,None
1045,NaN,NaN,NaN,5728886.0,STUDIO BANK,NASHVILLE,520930.0,TN,1.279647e+07,1.277762e+07,...,Nasdaq Capital Market,2023-05-16,cs,IEX_5638345442592D52,US,USD,True,BBG003BCFHX4,1531031.0,None


In [23]:
lender_holding_df_output = lender_holding_df_all.merge(final[['RSSDHCR','symbol','exchange','exchangeSegment']], on="RSSDHCR")

In [26]:
lender_holding_df_output = lender_holding_df_output[['RSSDHCR', 'OriginatingLender', 'OriginatingLenderCity', 'OriginatingLenderLocationID', 'OriginatingLenderState', 'CurrentApprovalAmount', 'ForgivenessAmount', 'ForgivnessPct', 'NAMEHCR', 'UNINUM', 'NM_LGL', 'NM_SHORT', 'CERT', 'symbol']]
lender_holding_df_output.rename({'symbol': 'ticker'}, inplace=True)
lender_holding_df_output

In [28]:
lender_holding_df_output.to_csv('../data/interim/lender_data.csv')

### Getting top holding company balance sheet data

In [85]:
USERNAME = ""
PASSWORD = ""

tiq = yq.Ticker(final.symbol.unique(), username = USERNAME, password = PASSWORD)
shares = tiq.p_balance_sheet(frequency='q')
result = shares[['asOfDate','ShareIssued','OrdinarySharesNumber','PreferredSharesNumber','TotalDebt','PreferredSharesNumber','PreferredStock','PreferredStockEquity', 'TotalDebt','CashAndCashEquivalents','CashFinancial','CashEquivalents','NetDebt', 'NetTangibleAssets']]
# result.reset_index(inplace=True)
# result

In [88]:
result.to_csv('../data/interim/holding_balance_sheet.csv')

### Getting stock price data

In [ ]:
def get_stock_data_from_tickers(tickers, start='2020-03-01', end='2020-03-01'):
    collection = []
    [collection.append(fetch_stock_history(x, start=start, end=end)) for x in tickers]
    return pd.concat(collection)

def fetch_stock_history(ticker, start, end):
    stock = yf.Ticker(ticker)
    df_stock = stock.history(start='2022-03-01', end='2022-12-31')
    df_stock['symbol'] = ticker
    df_stock.reset_index(inplace=True)
    return df_stock
    
result = get_stock_data_from_tickers(final.symbol.unique())

result.to_csv('../data/interim/stocks.csv')

# EXPERIMENTAL

## Fuzzy match to get closer ticker matches (takes hours to run)

In [216]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1
final2 = fuzzy_merge(lender_holding_df, tickers, 'NM_LGL', 'name', threshold=80)



### Getting CIK identifier

In [6]:
# Testing getting cik
# TODO: Return all cik's if SEC returns multiple matches (multiple search currently fails)

import requests
from bs4 import BeautifulSoup

def get_cik_from_company_name(company_name):
    
    # Setting up request
    url = "https://www.sec.gov/cgi-bin/browse-edgar"
    params = {
        'company': company_name,
        'match': 'contains',
        'owner': 'exclude',
    }
    headers = {
        'User-Agent' : '' # SEC expects '[Name] [Email]' on this header per SEC developer's policy https://www.sec.gov/os/webmaster-faq#developers
    }
    

    # Sending request and parsing
    response = requests.get(url, params=params, headers=headers)
    parsable = BeautifulSoup(response.text, 'html.parser')
    cik_element = parsable.find('acronym', string='CIK')
    cik = cik_element.cik = cik_element.find_next_sibling('a').text if cik_element else None if cik_element else None

    return cik

# Testing the function
get_cik_from_company_name('GOLDMAN SACHS BANK AG')

AttributeError: 'NoneType' object has no attribute 'cik'

### Other: EDA

In [9]:
# What is the percentage of $ value lent that is forgiven?
grp = PPP_df[['CurrentApprovalAmount','ForgivenessAmount']].agg(['sum']).reset_index()
grp['ForgivnessPct'] = grp.ForgivenessAmount / grp.CurrentApprovalAmount
grp

,index,CurrentApprovalAmount,ForgivenessAmount,ForgivnessPct
0,sum,7.896639e+11,7.571097e+11,0.958775


In [12]:
PPP_df

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,BusinessType,OriginatingLenderLocationID,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit,ForgivenessAmount,ForgivenessDate
0,0,9547507704,05/01/2020,464.0,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,NaN,29150-9662,...,Corporation,19248.0,Synovus Bank,COLUMBUS,GA,Unanswered,Unanswered,NaN,773553.37,11/20/2020
1,1,9777677704,05/01/2020,464.0,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,NaN,29420-9000,...,Sole Proprietorship,19248.0,Synovus Bank,COLUMBUS,GA,Male Owned,Non-Veteran,NaN,746336.24,08/12/2021
2,2,5791407702,05/01/2020,1013.0,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,NaN,98112-2922,...,Non-Profit Organization,9551.0,"Bank of America, National Association",CHARLOTTE,NC,Unanswered,Unanswered,Y,696677.49,02/10/2021
3,3,6223567700,05/01/2020,920.0,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,SAN BERNARDINO,NaN,92407-1740,...,Corporation,9551.0,"Bank of America, National Association",CHARLOTTE,NC,Male Owned,Non-Veteran,NaN,395264.11,09/10/2021
4,4,9662437702,05/01/2020,101.0,PPP,AERO BOX LLC,NaN,NaN,NaN,NaN,...,NaN,57328.0,The Huntington National Bank,COLUMBUS,OH,Unanswered,Unanswered,NaN,370819.35,04/08/2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11460470,899995,5915717301,04/30/2020,1086.0,PPP,IN S KANG,13422 SW 128TH PL,TIGARD,OR,97223,...,Sole Proprietorship,456756.0,Cross River Bank,TEANECK,NJ,Unanswered,Unanswered,NaN,9732.15,01/07/2021
11460471,899996,8795397408,05/19/2020,1086.0,PPP,TRINAMIC STUDIO,14680 BRICKYARD DR SW,SHERWOOD,OR,97140,...,Limited Liability Company(LLC),53803.0,"U.S. Bank, National Association",CINCINNATI,OH,Unanswered,Unanswered,NaN,9766.62,05/27/2021
11460472,899997,6652287305,04/30/2020,1086.0,PPP,REFUGE COMMUNITY CHURCH,3519 NE 15th Ave Ste 554,PORTLAND,OR,97212-2356,...,Non-Profit Organization,103851.0,Northwest Community CU,EUGENE,OR,Unanswered,Unanswered,Y,9827.43,01/04/2022
11460473,899998,7915838308,01/28/2021,1086.0,PPS,IN S KANG,13422 SW 128th Pl,Tigard,OR,97223-2848,...,Sole Proprietorship,456756.0,Cross River Bank,TEANECK,NJ,Unanswered,Unanswered,NaN,9729.29,10/06/2021
